In [ ]:
##I run the following cells for each year individually and change the year as I go

import numpy as np
import scipy as sp
import pandas as pd
import os
import csv
year = "2022"
path = f"C:/Users/asobc/Harris-Public-Policy Dropbox/Alex Sobczynski/WorldBank_AFG_Climate/data/GLDAS_Processed/gldas_processed_{year}"
path_big = f"C:/Users/asobc/Harris-Public-Policy Dropbox/Alex Sobczynski/WorldBank_AFG_Climate/data/GLDAS/csv_{year}/"
files = os.listdir(path_big)
csv_files = [file for file in files if file.endswith('.csv')]
if csv_files:
    # Get the first CSV file
    first_csv_file = csv_files[0]
    print(first_csv_file)
    # Load the first CSV file into a pandas DataFrame
    data = pd.read_csv(os.path.join(path_big, first_csv_file), encoding='latin-1')

    # Display the loaded data
    print(data.columns)
else:
    print("No CSV files found in the folder.")

In [ ]:
#for each file in a year, I want to pull the variable that starts with GWS or TWS
#then I want to bind it by grid cell to all the others that year (add if statement that confirms lon and lat values match before binding)
#then I want to calculate the mean and variance across the rows and put them in their own columns
#add a year column
#add a distid column
#make a panel by binding
#doing diagnostics
columns_to_extract = ['Unnamed: 0', 'lon', 'lat'] + [col for col in data.columns if col.startswith('TWS')]
df_combined = data[columns_to_extract]
print(df_combined.head(2))

In [ ]:
for file_name in os.listdir(path_big):
    if file_name.endswith('.csv') and file_name != first_csv_file:
        file_path = os.path.join(path_big, file_name)
        df = pd.read_csv(file_path)

        # Extract the columns that start with 'TWS' or 'GWS'
        columns_to_bind = [col for col in df.columns if col.startswith('TWS')]

        # Merge the DataFrame with the previously created DataFrame based on 'lon' and 'lat'
        df_combined = pd.merge(df_combined, df[['lon', 'lat'] + columns_to_bind], on=['lon', 'lat'], how='outer')
print(df_combined.head(2))

In [ ]:
length = df_combined.shape[1]
last_columns = df_combined.columns[3:length]
df_combined['annual_mean'] = df_combined[last_columns].mean(axis=1)
df_combined['annual_variance'] = df_combined[last_columns].var(axis=1)
df_combined['annual_95th_pctl'] = df_combined[last_columns].quantile(0.95, axis=1)
df_combined['annual_5th_pctl'] = df_combined[last_columns].quantile(0.05, axis=1)

print(df_combined['annual_mean'].head(5))
print(df_combined.shape)
df_summ = df_combined.iloc[:, :3].join(df_combined.iloc[:, -4:])
print(df_summ.head(4))
df_summ.rename(columns={'annual_mean_TWS': 'annual_mean'}, inplace=True)
df_summ.rename(columns={'annual_variance': 'annual_variance_TWS'}, inplace=True)
df_summ.rename(columns={'annual_95th_pctl': 'annual_95th_pctl_TWS'}, inplace=True)
df_summ.rename(columns={'annual_5th_pctl': 'annual_5th_pctl_TWS'}, inplace=True)
df_summ.rename(columns={'Unnamed: 0': 'cell_id'}, inplace=True)
df_summ['year'] = year
df_summ.to_csv(f'{year}_summary_TWS.csv', index=False)

Now I'm going to merge everything.

In [33]:
# Here's the code to merge everything
import numpy as np
import scipy as sp
import pandas as pd
import os
import csv

clean_path = "C:/Users/asobc/Harris-Public-Policy Dropbox/Alex Sobczynski/WorldBank_AFG_Climate/data/cleaning/"
clean_files = os.listdir(clean_path)
sum_files = [file for file in clean_files if file.endswith('TWS.csv')]

#
# dfs = []
#
# # Read each CSV file into a DataFrame and append to dfs list
# for file in clean_files:
#     df = pd.read_csv(os.path.join(clean_path, file))
#     dfs.append(df)
#
# # Concatenate DataFrames along columns
# combined_df = pd.concat(dfs, axis=1)
#
# # Write the combined DataFrame to a new CSV file
# combined_df.to_csv("groundwater_cbind.csv", index=False)

Now I want to make a raster file (with standardized color scale) for each groundwater file.

In [ ]:
check = pd.read_csv(f'{clean_path}2010_summary_TWS.csv')

max_value = check['annual_variance'].max()
print(max_value)

In [ ]:
import rasterio
import numpy as np
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
import os

sum_files = [file for file in clean_files if file.endswith('TWS.csv')]
out_path = "C:/Users/asobc/Harris-Public-Policy Dropbox/Alex Sobczynski/WorldBank_AFG_Climate/output/groundwater/TWS pictures/"
little = {
    'name': ["annual_mean", 'annual_variance', 'annual_95th_pctl','annual_5th_pctl' ],
    'type': ['Average', 'Variance', '95th Percentile','5th Percentile'],
    'short': ['mean', 'var', '95th','5th']
}
df_little = pd.DataFrame(little)
for index, row in df_little.iterrows():
    name = row['name']
    type = row['type']
    short = row['short']
    for file_name in sum_files:
        year2 = file_name[:4]
        file_path = os.path.join(clean_path, file_name)
        my_data = pd.read_csv(file_path)
        lon = my_data['lon']
        lat = my_data['lat']
        vector = my_data[name] #fill in my_vector

        # Set up the raster properties
        pixel_size = 0.25
        x_min, x_max = lon.min(), lon.max()
        y_min, y_max = lat.min(), lat.max()

        cols = int((x_max - x_min) / pixel_size)
        rows = int((y_max - y_min) / pixel_size)

        # Create a raster
        transform = from_origin(x_min, y_max, pixel_size, pixel_size)
        raster = rasterio.open(f'{clean_path}{year2}_{short}_TWS.tif', 'w', driver='GTiff', height=rows, width=cols, count=1, dtype='float32', crs='EPSG:4326', transform=transform) #fill in path

        # Convert lon, lat, and vector data to a numpy array
        array = np.zeros((rows, cols), dtype=np.float32)

        for i in range(len(lon)):
            col = int((lon[i] - x_min) / pixel_size)
            row = int((y_max - lat[i]) / pixel_size)

            if 0 <= row < rows and 0 <= col < cols:
                array[row, col] = vector[i]

        raster.write(array, 1)
        raster.close()

        with rasterio.open(f'{clean_path}{year2}_{short}_TWS.tif') as src: #same path as above
            # Read the raster data
            raster_data = src.read(1)
            raster_data = np.where(raster_data == src.nodata, 0, raster_data)
            min_value = np.nanmin(raster_data)
            max_value = np.nanmax(raster_data)
        plt.figure(figsize=(12, 12))
        plt.imshow(raster_data, cmap='Blues',vmin=0, vmax=1500)
        plt.title(year2)
        plt.colorbar(label=f'{type} Terrestrial Water Storage (mm)')
        plt.axis('off')
        # plt.show()
        # Add legend and title
        # plt.text(1, 0.5, f'Min: {min_value}, Max: {max_value}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)
        # plt.text(0.5, 1.05, 'Legend and Title', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

        # Save the plot as a PNG
        plt.savefig(f'{out_path}{year2}_{short}_TWS_legend.png', bbox_inches='tight', pad_inches=0.1)
        print(f'saved {year2}_{short}_TWS_legend.png')
        plt.close()

In [ ]:
import matplotlib as mpl
import rasterio
import matplotlib.pyplot as plt
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']
for year in years:
    with rasterio.open(f'{clean_path}{year}_raster.tif') as src: #same path as above
        # Read the raster data
        raster_data = src.read(1)
        raster_data = np.where(raster_data == src.nodata, 255, raster_data)
        min_value = np.nanmin(raster_data)
        max_value = np.nanmax(raster_data)
    plt.figure(figsize=(12, 12))
    plt.imshow(raster_data, cmap='Blues',vmin=0, vmax=1500)
    plt.title(year)
    plt.colorbar(label='Average Groundwater Storage (mm)')
    plt.axis('off')
    #plt.show()
    # Add legend and title
    #plt.text(1, 0.5, f'Min: {min_value}, Max: {max_value}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)
    #plt.text(0.5, 1.05, 'Legend and Title', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

    # Save the plot as a PNG
    plt.savefig(f'{clean_path}{year}_raster_with_legend.png', bbox_inches='tight', pad_inches=0.1)

Make a GIF

In [ ]:
import imageio
import imageio.v2 as imageio
GWS_path = "C:/Users/asobc/Harris-Public-Policy Dropbox/Alex Sobczynski/WorldBank_AFG_Climate/output/groundwater/GWS pictures/"
GWS_files = os.listdir(GWS_path)
# Filter only files with certain extensions (e.g., PNG, JPG, etc.)
image_files = sorted([file for file in GWS_files if file.endswith('_5th_GWS_legend.png')])
#image_files = sorted([file for file in GWS_files if file.endswith('_95th_GWS_legend.png') and '2010' not in file])
# Initialize a list to store images
images = []

# Read each image file and append to the list
for filename in image_files:
    filepath = os.path.join(GWS_path, filename)
    images.append(imageio.imread(filepath))

# Create a GIF from the list of images
gif_path = f'{GWS_path}5th_GWS.gif'
imageio.mimsave(gif_path, images, duration = 1, fps = 1, loop = 0)

#imageio.mimwrite(gif_path, images, duration = 1, loop = 0)

print(f"GIF saved to: {gif_path}")

In [ ]:
#I want to take a look at the data to get an idea for seasonality
import matplotlib.pyplot as plt

df2 = df_combined.drop(df_combined.columns[[1, 2]], axis=1)
df2.set_index('Unnamed: 0', inplace=True)  # Setting the index to Place column
df2 = df2.iloc[:, ::2]

df2 = df2.T
# Randomly select 10 rows
random_rows = df2.sample(n=10, axis = 1)

# Plotting
plt.figure(figsize=(8, 6))
for i, row in enumerate(random_rows.iterrows()):
    place, data = row
    plt.plot(df2.index, df2.values, label=place)

plt.xlabel('Day')
plt.ylabel('Value')
plt.title('Randomly Selected Places Over Time')
plt.legend()
plt.grid(True)
plt.show()